In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy import ndimage

from tqdm import tqdm_notebook
import kaggle_util

In [2]:
def get_mask(encodepixel):
    shape = (768,768)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    masks = encodepixel
    
#     print(img_id, masks)
    
    if(type(masks.iloc[0]) == float): return img.reshape(shape)
    for mask in masks:
        s = mask.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1
    return img.reshape(shape).T

In [3]:
one_result = pd.read_csv('../result/XUnet256_0_0677.csv')
ship_detect = pd.read_csv('../result/ship_detection.csv')

In [8]:
ship_detect.rename({'id':'ImageId'}, axis=1, inplace=True)

In [10]:
df_all = pd.merge(one_result, ship_detect, on='ImageId', how='left')

In [12]:
df_all.fillna('', inplace=True)

In [15]:
df_all.loc[(df_all.EncodedPixels.str.len()>0) & (df_all.p_ship < 0.5)].shape

(66, 3)

In [28]:
def split_mask(mask):
    threshold = 0.5
    threshold_obj = 0 #ignor predictions composed of "threshold_obj" pixels or less
    labled,n_objs = ndimage.label(mask)
    result = []
    for i in range(n_objs):
        obj = (labled == i + 1).astype(int)
        if(obj.sum() > threshold_obj): result.append(obj)
    return result

def decode_mask(mask, shape=(768, 768)):
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [29]:
new_result

,ImageId,EncodedPixels


In [9]:
one_result.fillna('', inplace=True)

In [42]:
new_result = pd.DataFrame(columns=one_result.columns)
cnt = 0
for idx, row in tqdm_notebook(one_result.iterrows()):
    if len(row.EncodedPixels) == 0:
        new_result = new_result.append(row, ignore_index=True)
    else:
        mask = get_mask(one_result.loc[one_result.ImageId == row.ImageId, 'EncodedPixels'])
        results = split_mask(mask)
        
        for msk in results:
            new_result = new_result.append({'ImageId':row.ImageId, 'EncodedPixels': decode_mask(msk)}, ignore_index=True)
#         cnt += 1
#         if cnt > 10:
#             break

In [43]:
kaggle_util.save_result(new_result, '../result/XUnet256_split.csv', competition = 'airbus-ship-detection', send = True, index = False)

save result
upload result
cmd: kaggle competitions submit -c airbus-ship-detection -f ../result/XUnet256_split.csv.7z -m "submit"
